In [97]:
import pandas as pd
import os 
from datetime import datetime, timedelta, date 
import yfinance as yfin
import numpy as np

In [5]:
aggregate = pd.read_pickle('../data/aggregate.pkl')
aggregate


,file_from,date,stock,tick,meeting_date,close,index_id
0,five_0035.pkl,2017-07-26,NYSE,DFS,2017-07-26,56.876797,1
1,five_0035.pkl,2017-07-26,OTCPK,LVMHF,2017-07-26,233.518677,3
2,five_0035.pkl,2017-07-26,NYSE,KN,2017-07-26,16.340000,4
3,five_0035.pkl,2017-07-26,NYSE,EW,2017-07-26,38.536667,5
4,five_0035.pkl,2017-07-26,NASDAQ,TSCO,2017-07-26,48.847878,6
...,...,...,...,...,...,...,...
139411,five_0165.pkl,2013-08-06,NYSE,TGH,2013-08-06,27.590778,9986
139412,five_0165.pkl,2013-08-06,NYSE,EFC,2013-08-06,8.335198,9988
139413,five_0165.pkl,2013-08-06,NYSE,WMC,2013-08-06,140.625427,9989
139414,five_0165.pkl,2019-04-18,NYSE,URI,2019-04-18,136.000000,9998


In [4]:
# DO NOT RUN if run aggregate.pkl. Since it is alredy pickle-d so format is kept
aggregate['file_from'] = aggregate['file_from'].astype('category')
aggregate['stock'] = aggregate['stock'].astype('category')
aggregate['date'] = pd.to_datetime(aggregate['date'], format='%Y-%m-%d')
aggregate['meeting_date'] = pd.to_datetime(aggregate['meeting_date'], format='%Y-%m-%d')

In [6]:
#aggregate.to_pickle('../data/aggregate.pkl')

In [6]:

tick_meeting_date = aggregate[['tick', 'meeting_date']]

In [7]:


def get_closing_data(ticker: str='AAPL', start_date: date = datetime(2020,2,2), meeting_date: date = datetime(2023,2,2) ) -> pd.DataFrame:
    '''
    Example: 
    ticker: 'AAPL'
    meeting_date: datetime.date, '2020-01-01' 
    id_vars = datetime.date 
    var names: close, DMA50, and DMA200 from wide to long format
    value: simple moving average and close price of ticker
    returns: pd.melted dataframe 

    
    '''
    days_go_back = timedelta(700)
    two_days_later = timedelta(20)

    #m_day = start_date.date()

    start = start_date - days_go_back
    end = meeting_date + two_days_later
    one_day = '1d'

    ticker = yfin.Ticker(ticker)

    data = ticker.history(start=start,  end=end, period=one_day)[['Close', 'Volume']]
    data = data.reset_index(inplace=False)
    data['Date'] = data['Date'].apply(lambda x: x.date())
    
    # Add features: simple moving average 50 vs. 200 days -> for smooting
    data['50DMA'] = data['Close'].rolling(50).mean()
    data['200DMA'] = data['Close'].rolling(200).mean()

    print(f'start date: {start}')
    print(f'meeting date: {meeting_date}')
    print(f'ending date: {end}')
    print(f'Ticker: {ticker}')
    
    data_table = data[['Date', 'Close', '50DMA', '200DMA']] 

    data_table = data_table.dropna()

    
    data_melted = pd.melt(data_table, id_vars='Date', var_name='Type', value_name='Price')

    return data_melted


get_closing_data(ticker='F')

start date: 2018-03-04 00:00:00
meeting date: 2023-02-02 00:00:00
ending date: 2023-02-22 00:00:00
Ticker: yfinance.Ticker object <F>


,Date,Type,Price
0,2018-12-17,Close,6.940988
1,2018-12-18,Close,6.916491
2,2018-12-19,Close,6.794002
3,2018-12-20,Close,6.745008
4,2018-12-21,Close,6.573524
...,...,...,...
3151,2023-02-14,200DMA,12.049801
3152,2023-02-15,200DMA,12.049203
3153,2023-02-16,200DMA,12.048013
3154,2023-02-17,200DMA,12.045419


In [8]:
# https://plotly.com/python/horizontal-vertical-shapes/

In [8]:
import plotly.graph_objects as go

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_usa_states.csv')

print(df.head())

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='center'),
    cells=dict(values=df.transpose().values.tolist(),
               fill_color='lavender',
               align='center'))
])

fig.show()

   Rank       State Postal  Population
0     1     Alabama     AL   4849377.0
1     2      Alaska     AK    736732.0
2     3     Arizona     AZ   6731484.0
3     4    Arkansas     AR   2966369.0
4     5  California     CA  38802500.0


In [9]:
top1200 = (aggregate.groupby('tick').size()
      .sort_values(ascending=False)
      .head(1200))


In [10]:
def get_single_ticker(tick: str='DFS') -> pd.DataFrame:
    data = tick_meeting_date[tick_meeting_date['tick'] == tick].copy()
    data = data.sort_values(by='meeting_date')
    data.reset_index(drop=True, inplace=True)
    
    min_date = data.meeting_date.min()
    max_date = data.meeting_date.max()
    
    closing = get_closing_data(ticker=tick, start_date=min_date.date(), meeting_date = max_date.date() )
    
    return closing

In [11]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import time

app = Dash(__name__)



app.layout = html.Div([
    html.H4('Stock price analysis'),
    dcc.Graph(id="time-series-chart"),
    html.P("Select stock:"),
    dcc.Dropdown(
        id="ticker",
        options=top1200.index,
        value='AAPL',
        clearable=False,
    ),
])


@app.callback(
    Output("time-series-chart", "figure"), 
    Input("ticker", "value"))
def display_time_series(ticker):
    
    meeting = tick_meeting_date[tick_meeting_date['tick'] == ticker]
    meeting_list = meeting.sort_values(by='meeting_date')['meeting_date'].to_list()
    

    data = get_single_ticker(ticker)
    fig = px.line(data, x='Date', y='Price', color='Type')
    # Set title
    fig.update_layout(
        title_text=f"Time series with range slider and selectors: {ticker}"
    )

    # Add range slider
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                        label="1m",
                        step="month",
                        stepmode="backward"),
                    dict(count=6,
                        label="6m",
                        step="month",
                        stepmode="backward"),
                    dict(count=1,
                        label="YTD",
                        step="year",
                        stepmode="todate"),
                    dict(count=1,
                        label="1y",
                        step="year",
                        stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )
    for mdate in meeting_list:
        fig.add_vline(x=mdate, line_dash='dash', line_color='green', line_width=.5)

    return fig


app.run_server(debug=True)

In [12]:
ticker_list = pd.unique(aggregate.tick)
len(ticker_list)

6198

In [13]:
records = (aggregate.groupby('tick').size()
      .sort_values(ascending=False)
)

In [14]:
# already saved in csv format
pd.DataFrame({
    'Ticker':records.index,
    'EarningsCount':records.values})#.to_csv('../data/ticker_earnings_count.csv', index=False)

,Ticker,EarningsCount
0,T,139
1,F,127
2,V,126
3,CSCO,119
4,MA,115
...,...,...
6192,TRX,1
6193,TRYIY,1
6194,HPOL,1
6195,HNHAF,1


In [228]:
import plotly.graph_objects as go

df = pd.read_csv('../data/ticker_30plus_custom.csv', encoding='utf-8')


fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='center'),
    cells=dict(values=df.transpose().values.tolist(),
               fill_color='lavender',
               align='center'))
])

fig.show()

In [17]:
ticker_earnings = pd.read_csv('../data/ticker_earnings_count.csv')

In [88]:
ticker_earnings = ticker_earnings[ticker_earnings['EarningsCount']>=30]

In [102]:
# values needed = ['longName', 'country', 'industry', 'marketCap', 'exchange']

def ticker_more_info(dataframe: pd.DataFrame=None):
    output = {'Ticker':[], 'LongName':[], 'Country':[], \
        'Industry':[], 'MarketCap':[], 'Exchange':[]}
    for ind, r in enumerate(dataframe.to_dict('records')):
        
        tick = yfin.Ticker(r['Ticker'])
        tick_info = tick.get_info()
        print(f"printing {ind, tick_info['symbol']}")
        
        try:
            output['Ticker'].append(tick_info['symbol'])
        except KeyError:
            output['LongName'].append(np.nan)

        try:
            output['LongName'].append(tick_info['longName'])
        except KeyError:
            output['LongName'].append(np.nan)

        try:
            output['Country'].append(tick_info['country'])
        except KeyError:
            output['Country'].append(np.nan)
        
        try:
            output['Industry'].append(tick_info['industry'])
        except KeyError:
            output['Industry'].append(np.nan)
        
        try:
            output['MarketCap'].append(tick_info['marketCap'])
        except KeyError:
            output['MarketCap'].append(np.nan)
        
        try:
            output['Exchange'].append(tick_info['exchange'])
        except KeyError:
            output['Exchange'].append(np.nan)
    
    return pd.DataFrame(output)

earnings_30plus_extended = ticker_more_info(dataframe=ticker_earnings)

printing (0, 'T')
printing (1, 'F')
printing (2, 'V')
printing (3, 'CSCO')
printing (4, 'MA')
printing (5, 'MU')
printing (6, 'GILD')
printing (7, 'BMY')
printing (8, 'MSFT')
printing (9, 'AMGN')
printing (10, 'JNJ')
printing (11, 'REGN')
printing (12, 'NVDA')
printing (13, 'GM')
printing (14, 'MMM')
printing (15, 'GE')
printing (16, 'VZ')
printing (17, 'INTC')
printing (18, 'PFE')
printing (19, 'CMCSA')
printing (20, 'LLY')
printing (21, 'AMD')
printing (22, 'WFC')
printing (23, 'MRK')
printing (24, 'MDT')
printing (25, 'SIRI')
printing (26, 'WDC')
printing (27, 'EXEL')
printing (28, 'GLW')
printing (29, 'HON')
printing (30, 'IBM')
printing (31, 'WMT')
printing (32, 'HD')
printing (33, 'ABBV')
printing (34, 'HPQ')
printing (35, 'SRPT')
printing (36, 'AMT')
printing (37, 'TXN')
printing (38, 'QCOM')
printing (39, 'PG')
printing (40, 'AMAT')
printing (41, 'JPM')
printing (42, 'MNKD')
printing (43, 'AMRN')
printing (44, 'USB')
printing (45, 'LRCX')
printing (46, 'KO')
printing (47, 'EOG'

In [106]:
#np.split_array(ticker_earnings, 4)
earnings_30plus_extended

In [157]:
aggregate[aggregate['tick'] == 'KBAL'].head()

,file_from,date,stock,tick,meeting_date,close,index_id
8326,five_0155.pkl,2020-02-05,noInfo,KBAL,2020-02-05,15.591878,1755
8906,five_0155.pkl,2019-05-11,noInfo,KBAL,2019-05-08,15.071750,2655
9887,five_0155.pkl,2017-05-06,noInfo,KBAL,2017-05-04,16.250475,4086
24664,five_0025.pkl,2016-05-04,noInfo,KBAL,2016-05-04,10.222046,5515
27184,five_0025.pkl,2018-02-01,noInfo,KBAL,2018-02-01,16.133982,9147


In [113]:
pd.read_pickle('../data/Six/six_0035.pkl')[pd.read_pickle('../data/Six/six_0035.pkl')['tick'] =='HSC']

,id,url,transcript,recorded_date,date,tagged,stock_ticker,stock,tick,meeting_date,close,volume,sma50_close,sma200_close,sma50_vol,sma200_vol,clean
1503,1503,https://seekingalpha.com/article/2039513-harsc...,[Harsco (HSC) Q4 2013 Earnings Call February 2...,"Fri, Feb. 21, 2014",2014-02-21,Harsco (HSC) Q4 2013 Earnings Call February 21...,"[noInfo, HSC]",noInfo,HSC,2014-02-21,23.148294,583243.0,23.877436,22.809548,382175.76,432780.495,"[harsco, hsc, earn, februari, execut, kenneth,..."


In [132]:
records = pd.read_csv('../data/ticker_30plus_records.csv')

In [136]:
def manual_entry_infoNotFound(dataframe: pd.DataFrame=None):
    for ind, r in enumerate(dataframe.to_dict('records')):
        tick = r['Ticker']
        print(tick)


manual_entry_infoNotFound(dataframe=records)

T
F
V
CSCO
MA
MU
GILD
BMY
MSFT
AMGN
JNJ
REGN
NVDA
GM
MMM
GE
VZ
INTC
PFE
CMCSA
LLY
AMD
WFC
MRK
MDT
SIRI
WDC
EXEL
GLW
HON
IBM
WMT
HD
ABBV
HPQ
SRPT
AMT
TXN
QCOM
PG
AMAT
JPM
MNKD
AMRN
USB
LRCX
KO
EOG
CCI
UNP
ANET
VMW
BA
CAT
NOW
AXP
CRM
TMUS
BIIB
SBUX
ILMN
PM
STX
PANW
CSX
DIS
CL
RY
TD
AA
CVS
BX
AIG
NXPI
M
EBAY
NVAX
MDLZ
BNS
VRTX
DHR
MCD
WDAY
AFL
C
FTNT
ALNY
NFLX
LOW
DAL
GOOG
TMO
BSX
AGNC
ACAD
FCX
SPLK
MMP
NOC
UAL
ABT
CLX
ASML
CMI
COP
LMT
SWKS
DDD
IRM
MO
CTSH
A
GIS
CAH
ADSK
PSX
STZ
TEVA
SNY
XOM
JNPR
ADI
NBIX
SGMO
KKR
RIG
WMB
GSK
SAP
SYK
BAC
ADM
PYPL
PRU
BLK
ALKS
MOS
FOLD
KMB
CVX
ALB
PEP
WM
NGD
IMGN
MET
MAR
ISRG
LUV
HAL
MRNA
BDX
FRT
PLUG
CALX
CHD
CCJ
EBS
HCA
CMG
INCY
GPS
RMBS
CHKP
AAL
ADP
NVO
ADBE
NVS
MELI
IVR
WAT
IMAX
CRIS
ARCC
LXRX
BMO
HIG
INTU
HOV
EA
LYB
CYTK
KR
NEM
CHTR
WY
WPC
TAP
SWN
BAX
APD
NEE
SWK
AB
HALO
ABC
SKT
YELP
PXD
WWE
JBLU
SGEN
TTWO
SLF
NTAP
STT
MRVL
SSYS
OLED
CVGI
DASTY
HL
DG
UNM
XRX
MCK
BBY
TSLA
AEM
FITB
DUK
TEX
GRPN
IDCC
AMKR
RF
RGLD
ULTA
FFIV
MTH
DPZ
EXAS
STM
CNSL
CNMD
MFC


In [139]:
flws = yfin.Ticker('INT')
for k, v in flws.get_info().items():
    print(k, v)

quoteType NONE
symbol INT
underlyingSymbol INT
uuid 53f93baa-3057-3211-87c7-50323892fa92
maxAge 86400
trailingPegRatio None


In [140]:
def find_ticker(ticker: str=None):
    return aggregate[aggregate['tick'] == ticker]

In [142]:
find_ticker(ticker='INT').head()

,file_from,date,stock,tick,meeting_date,close,index_id
2280,five_0035.pkl,2015-08-03,noInfo,INT,2015-07-30,44.180782,3149
7535,five_0155.pkl,2020-10-31,noInfo,INT,2020-10-29,18.469917,666
9425,five_0155.pkl,2015-02-12,noInfo,INT,2015-02-12,48.017761,3453
10768,five_0155.pkl,2017-07-27,noInfo,INT,2017-07-27,37.425262,5429
17655,five_0145.pkl,2013-02-21,noInfo,INT,2013-02-21,40.762051,5748


In [212]:
def check_nasdaq_listed(ticker: str=None):
    nasdaq_listed = pd.read_csv('../data/nasdaq/nasdaq_listed.csv')[['Symbol', 'Security Name']]
    
    try:
        tick = nasdaq_listed[nasdaq_listed['Symbol'] == ticker]['Security Name'].iloc[0]
        return tick
    except IndexError:
        return np.nan

def check_nasdaq_screener(ticker: str=None):
    nasdaq_screener = pd.read_csv('../data/nasdaq/nasdaq_screener.csv')[['Symbol','Name']]
    try:
        tick = nasdaq_screener[nasdaq_screener['Symbol'] == ticker]['Name'].iloc[0]
        return tick
    except IndexError:
        return np.nan

def check_nasdaq_traded(ticker: str=None):
    nasdaq_traded = pd.read_csv('../data/nasdaq/nasdaq_traded.csv')[['Symbol', 'Security Name']]
    try:
        tick = nasdaq_traded[nasdaq_traded['Symbol'] == ticker]['Security Name'].iloc[0]
        return tick
    except IndexError:
        return np.nan

check_nasdaq_traded(ticker='AACI')


'Armada Acquisition Corp. I - Common Stock'

In [211]:
def check_nyse_listed(ticker: str=None):
    nyse_listed = pd.read_csv('../data/nyse_other_listings/data/nyse-listed_csv.csv')[['ACT Symbol', 'Company Name']]
    
    try:
        tick = nyse_listed[nyse_listed['ACT Symbol'] == ticker]['Company Name'].iloc[0]
        return tick
    except IndexError:
        return np.nan
    
def check_nyse_other(ticker: str=None):
    nyse_other = pd.read_csv('../data/nyse_other_listings/data/other-listed_csv.csv')[['ACT Symbol', 'Company Name']]
    
    try:
        tick = nyse_other[nyse_other['ACT Symbol'] == ticker]['Company Name'].iloc[0]
        return tick
    except IndexError:
        return np.nan

In [188]:
pd.read_csv('../data/nyse_other_listings/data/other-listed_csv.csv').head()

,ACT Symbol,Company Name,Security Name,Exchange,CQS Symbol,ETF,Round Lot Size,Test Issue,NASDAQ Symbol
0,A,"Agilent Technologies, Inc. Common Stock","Agilent Technologies, Inc. Common Stock",N,A,N,100.0,N,A
1,AA,Alcoa Inc. Common Stock,Alcoa Inc. Common Stock,N,AA,N,100.0,N,AA
2,AA$,Alcoa Inc. $3.75 Preferred Stock,Alcoa Inc. $3.75 Preferred Stock,A,AAp,N,100.0,N,AA-
3,AA$B,Alcoa Inc. Depository Shares Representing 1/10...,Alcoa Inc. Depository Shares Representing 1/10...,N,AApB,N,100.0,N,AA-B
4,AAC,"AAC Holdings, Inc. Common Stock","AAC Holdings, Inc. Common Stock",N,AAC,N,100.0,N,AAC


In [201]:
check_nyse_other(ticker='AA')

'Alcoa Inc. Common Stock'

In [209]:
records['nyse_other'] = records['Ticker'].apply(check_nyse_other)

In [214]:
records['nyse_listed'] = records['Ticker'].apply(check_nyse_listed)

In [218]:
records[records['Country'].isna()]

,Ticker,LongName,Country,Industry,MarketCap,Exchange,nyse_other,nyse_listed
53,CAT,Caterpillar Inc.,NaN,NaN,1.408334e+11,NYQ,"Caterpillar, Inc. Common Stock","Caterpillar, Inc. Common Stock"
211,DASTY,Dassault Systèmes SE,NaN,NaN,5.288333e+10,PNK,NaN,NaN
272,P,"Pandora Media, Inc.",NaN,NaN,NaN,YHD,"Pandora Media, Inc. Common Stock","Pandora Media, Inc. Common Stock"
273,BGCP,"BGC Partners, Inc.",NaN,NaN,1.734216e+09,NMS,NaN,NaN
356,PDCE,"PDC Energy, Inc.",NaN,NaN,6.424891e+09,NMS,NaN,NaN
...,...,...,...,...,...,...,...,...
2239,NVZMF,Novozymes A/S,NaN,NaN,1.206154e+10,PNK,NaN,NaN
2243,BTDPF,Barratt Developments plc,NaN,NaN,5.656366e+09,PNK,NaN,NaN
2244,CA,"CA, Inc.",NaN,NaN,NaN,YHD,NaN,NaN
2253,BNPQF,BNP Paribas SA,NaN,NaN,7.625635e+10,PNK,NaN,NaN


In [225]:
pd.read_csv('../data/ticker_30plus_custom.csv', encoding='utf-8')

,Ticker,LongName,Country,Industry,MarketCap,Exchange,Source
0,T,AT&T Inc.,United States,Telecom Services,1.008010e+11,NYQ,yfinance
1,F,Ford Motor Company,United States,Auto Manufacturers,4.790669e+10,NYQ,yfinance
2,V,Visa Inc.,United States,Credit Services,4.980290e+11,NYQ,yfinance
3,CSCO,"Cisco Systems, Inc.",United States,Communication Equipment,2.259210e+11,NMS,yfinance
4,MA,Mastercard Incorporated,United States,Credit Services,3.704780e+11,NYQ,yfinance
...,...,...,...,...,...,...,...
2253,BNPQF,BNP Paribas SA,France,Banking and Financial Services,7.625635e+10,PNK,https://group.bnpparibas/en/
2254,MARK,"Remark Holdings, Inc.",United States,SoftwareâInfrastructure,1.189161e+07,NCM,yfinance
2255,CABGY,Carlsberg A/S,Germany,Brewing,2.188818e+10,PNK,https://www.carlsberggroup.com/
2256,SR,Spire Inc.,United States,UtilitiesâRegulated Gas,3.098323e+09,NYQ,yfinance


In [226]:
records

,Ticker,LongName,Country,Industry,MarketCap,Exchange,nyse_other,nyse_listed
0,T,AT&T Inc.,United States,Telecom Services,1.008009e+11,NYQ,AT&T Inc.,AT&T Inc.
1,F,Ford Motor Company,United States,Auto Manufacturers,4.790669e+10,NYQ,Ford Motor Company Common Stock,Ford Motor Company Common Stock
2,V,Visa Inc.,United States,Credit Services,4.980289e+11,NYQ,Visa Inc.,Visa Inc.
3,CSCO,"Cisco Systems, Inc.",United States,Communication Equipment,2.259213e+11,NMS,NaN,NaN
4,MA,Mastercard Incorporated,United States,Credit Services,3.704778e+11,NYQ,Mastercard Incorporated Common Stock,Mastercard Incorporated Common Stock
...,...,...,...,...,...,...,...,...
2253,BNPQF,BNP Paribas SA,NaN,NaN,7.625635e+10,PNK,NaN,NaN
2254,MARK,"Remark Holdings, Inc.",United States,Software—Infrastructure,1.189161e+07,NCM,NaN,NaN
2255,CABGY,Carlsberg A/S,NaN,NaN,2.188818e+10,PNK,NaN,NaN
2256,SR,Spire Inc.,United States,Utilities—Regulated Gas,3.098323e+09,NYQ,Standard Register Company (The) Common Stock,Standard Register Company (The) Common Stock


In [227]:
pd.read_csv('../data/ticker_earnings_count.csv')

,Ticker,EarningsCount
0,T,139
1,F,127
2,V,126
3,CSCO,119
4,MA,115
...,...,...
6192,TRX,1
6193,TRYIY,1
6194,HPOL,1
6195,HNHAF,1


In [231]:

ticker_30plus = pd.read_csv('../data/ticker_30plus_custom.csv', encoding='utf-8')

In [236]:
ticker_30plus['EarningsCount']=ticker_earnings['EarningsCount'].to_list()

In [241]:
ticker_30plus[['Ticker', 'LongName', 'EarningsCount', 'Country', 'Industry', 'MarketCap', 'Exchange','Source']].to_csv('../data/ticker_30plus_custom.csv', index=False)

In [238]:
ticker_30plus.columns

Index(['Ticker', 'LongName', 'Country', 'Industry', 'MarketCap', 'Exchange',
       'Source', 'EarningsCount'],
      dtype='object')

In [246]:
import plotly.graph_objects as go

df = pd.read_csv('../data/ticker_30plus_custom.csv', encoding='utf-8')


fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='center'),
    cells=dict(values=df.transpose().values.tolist(),
               fill_color='lavender',
               align='center'))
])
fig.write_html('../data/results/earnings_30plus_custom.html')
fig.show()